In [1]:
import pandas as pd
import numpy as np
import math
import copy

## EDA

In [2]:
MOVIE=pd.read_csv("movies.csv")

In [3]:
MOVIE

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
Ratings=pd.read_csv("ratings.csv")

In [5]:
Ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
ratings=pd.merge(Ratings,MOVIE)

In [7]:
ratings.describe

<bound method NDFrame.describe of         userId  movieId  rating   timestamp                             title  \
0            1        1     4.0   964982703                  Toy Story (1995)   
1            5        1     4.0   847434962                  Toy Story (1995)   
2            7        1     4.5  1106635946                  Toy Story (1995)   
3           15        1     2.5  1510577970                  Toy Story (1995)   
4           17        1     4.5  1305696483                  Toy Story (1995)   
...        ...      ...     ...         ...                               ...   
100831     610   160341     2.5  1479545749                  Bloodmoon (1997)   
100832     610   160527     4.5  1479544998  Sympathy for the Underdog (1971)   
100833     610   160836     3.0  1493844794                     Hazard (2005)   
100834     610   163937     3.5  1493848789                Blair Witch (2016)   
100835     610   163981     3.5  1493850155                         31 (201

In [8]:
ratings.drop(['genres','timestamp'],axis=1)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [9]:
user_ratings=ratings.pivot_table(index=['userId'],columns=['title'],values='rating')

In [10]:


user_ratings

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


Here,NaN means that the user hasn't seen the movie and hasn't rated it.So,We will fill it with zeros.

In [11]:
user_ratings=user_ratings.dropna(thresh=10,axis=1).fillna(0) #Here we are dropping thoses movies which has been rated by less that 10 people

In [12]:
 UserRate = copy.deepcopy(user_ratings)

user_ratings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.5,3.5,0.0


## User to User based collaborative filtering

In [13]:
def similar_users(userID,user_ratings):
    rating=np.array(user_ratings)
    rows=rating.shape[0]
    cols=rating.shape[1]
    similar_users=[]
    for i in range(rows):
        if i!=userID-1:
            cosine_similarity=(np.dot(rating[i],rating[userID-1]))/(math.sqrt((rating[i]**2).sum())*math.sqrt((rating[userID-1]**2).sum())) #This will calculate the cosine similarity between User having id i+1 and userID
            user_similarity=[cosine_similarity,i]
            similar_users.append(user_similarity)
        
    return similar_users 

In [14]:
def movie_recommend(userID,user_ratings):
    similar_users_ID_similarity=similar_users(userID,user_ratings) #array containing the similarity of user havinf id userID with all the other users
    similar_users_decreasing_order=sorted(similar_users_ID_similarity,key=lambda l:l[0], reverse=True) #Arranging the similarity array in decreasing order
    most_similar_userID=similar_users_decreasing_order[0][1]+1 #Storing the most similar user found to user having id userID
    user_ratings.loc[most_similar_userID]=sorted(user_ratings.loc[most_similar_userID],reverse=True) #Sorting the movie rating of most similar user 
    recommended_movies=[]   #This list will contain all the movies recommended to user having id userID 
    for j in range(user_ratings.loc[most_similar_userID].shape[0]):
        
        if user_ratings.loc[most_similar_userID][j]!=0 and user_ratings.loc[userID][j]==0:
            if(user_ratings.loc[most_similar_userID][j]>=2.5):
                recommended_movies.append(user_ratings.columns[j])
    return recommended_movies

## In the below cell,id will store the value of userId for whom we are recommending movies

In [15]:
id=7
print(movie_recommend(id,user_ratings)) #This will give the list of movies recommended to userID=id.

["'burbs, The (1989)", '(500) Days of Summer (2009)', '10 Cloverfield Lane (2016)', '10 Things I Hate About You (1999)', '10,000 BC (2008)', '101 Dalmatians (1996)', '101 Dalmatians (One Hundred and One Dalmatians) (1961)', '12 Angry Men (1957)', '12 Years a Slave (2013)', '127 Hours (2010)', '13 Going on 30 (2004)', '13th Warrior, The (1999)', '1408 (2007)', '15 Minutes (2001)', '17 Again (2009)', '1984 (Nineteen Eighty-Four) (1984)', '2 Days in the Valley (1996)', '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)', '20,000 Leagues Under the Sea (1954)', '2010: The Year We Make Contact (1984)', '2012 (2009)', '21 (2008)', '21 Grams (2003)', '21 Jump Street (2012)', '22 Jump Street (2014)', '25th Hour (2002)', '27 Dresses (2008)', '28 Days (2000)', '28 Days Later (2002)', '28 Weeks Later (2007)', '30 Days of Night (2007)', '300 (2007)', '39 Steps, The (1935)', '3:10 to Yuma (2007)', '40 Days and 40 Nights (2002)', '40-Year-Old Virgin, The (2005)', '400 Blows, The (Les quatre cents

## Item Based collaborative filtering

In [16]:
from sklearn.metrics.pairwise import cosine_similarity #Used for calculating the similarity metrics between items
def recommend(userID,user_ratings):
    #This function will return the ratings of all unwatched movies of user having id userID
    similarity_matrix = np.array(cosine_similarity(user_ratings.T))#Calculates the similarity matrix of item to item
   
    user_ratings_array=np.array(user_ratings)
    
    user_index=userID-1
    cols=user_ratings_array.shape[1]
    for j in range(user_ratings_array.shape[1]):
        if user_ratings_array[user_index][j]==0:
            numerator=0
            denominator=0
            for i in range(cols):
                if i!=j and user_ratings_array[user_index][i]!=0: #made some changes after submission
                    numerator+=similarity_matrix[j][i]*user_ratings_array[user_index][i] 
                    denominator+=similarity_matrix[j][i]
            predicted_rating_for_jth_movie_by_userID=numerator/denominator
            user_ratings_array[user_index][j]=predicted_rating_for_jth_movie_by_userID
    return user_ratings_array[user_index]

In [17]:
def movies_name(userID,pivot_table):
    pivot_table.loc[userID]=recommend(userID,pivot_table)
    cols=pivot_table.shape[1]
    recommended_movies=[]
   
    for j in range(cols):
        if(pivot_table.loc[userID][j]>=3 and UserRate.loc[userID][j]==0 ): #Only recommending movies having rating greater than 3 and for which user hasn't given any rating 
            movies_and_ratings=[pivot_table.columns[j],pivot_table.loc[userID][j]]
            recommended_movies.append(movies_and_ratings)
    decreasing_order_recommended_movies=sorted(recommended_movies,key=lambda l:l[1], reverse=True) #sorting the recommended movies using rating
    return decreasing_order_recommended_movies
    

## In the below cell,id2 will store the value of userId for whom we are recommending movies

In [28]:
id2=11
print(movies_name(id2,user_ratings)) #This will give the list of movies recommended to userID 2 (id2 is added in the bracket later)

[['12 Angry Men (1957)', 3.8487828328788822], ['2001: A Space Odyssey (1968)', 3.817520297549683], ['American Beauty (1999)', 3.807550157032829], ['American History X (1998)', 3.8016616406361243], ["Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)", 3.800326076045671], ['28 Days Later (2002)', 3.796144860808957], ['Amadeus (1984)', 3.795150337434641], ['Alien (1979)', 3.7910707419280882], ['40-Year-Old Virgin, The (2005)', 3.7869481496353434], ['Aladdin (1992)', 3.786265875997999], ['Aliens (1986)', 3.785486706657506], ['300 (2007)', 3.7831844110288624], ['101 Dalmatians (One Hundred and One Dalmatians) (1961)', 3.781442311701491], ['American President, The (1995)', 3.7789513933071013], ['Apocalypse Now (1979)', 3.776886642619339], ['Back to the Future (1985)', 3.7744579169916843], ['Almost Famous (2000)', 3.7722423995104752], ['Babe (1995)', 3.7711665029294137], ['Beautiful Mind, A (2001)', 3.7711550381541015], ['African Queen, The (1951)', 3.7702709801286036], ['Adaptation (2002)

In [19]:
import jovian


In [20]:
jovian.commit()

    

<IPython.core.display.Javascript object>

[jovian] Updating notebook "aryanapopalghat/cs419-group12-code" on https://jovian.com/
[jovian] Committed successfully! https://jovian.com/aryanapopalghat/cs419-group12-code


'https://jovian.com/aryanapopalghat/cs419-group12-code'

In [21]:
jovian.commit(project="my-notebook", environment=None, outputs=None, notebook_id=None, files=None, message=None, git_commit=False, new_project=False, privacy='auto', filename=None, author=None, course=None, assignment=None, organization=None, tags=None)


[jovian] Error: "notebook_id" is deprecated. Use "project" instead.


<IPython.core.display.Javascript object>

[jovian] Updating notebook "aryanapopalghat/cs419-group12-code" on https://jovian.com/
[jovian] Committed successfully! https://jovian.com/aryanapopalghat/cs419-group12-code


'https://jovian.com/aryanapopalghat/cs419-group12-code'